In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Creating Augmented Datasets

In [2]:
# delete all the files and directories recursively in the current working directory ...

!rm -rf *

# make directory ...

!mkdir /kaggle/working/datasets_aug
!mkdir /kaggle/working/datasets_aug/badodd
!mkdir /kaggle/working/datasets_aug/badodd/labels
!mkdir /kaggle/working/datasets_aug/badodd/labels/train
!mkdir /kaggle/working/datasets_aug/badodd/images
!mkdir /kaggle/working/datasets_aug/badodd/images/train
!mkdir /kaggle/working/datasets_aug/badodd/images/test

In [3]:
def adjust_brightness(image, factor):
    return np.clip(image * factor, 0, 255).astype(np.uint8)

def adjust_contrast(image, factor):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.clip((image - gray.mean()) * factor + gray.mean(), 0, 255).astype(np.uint8)

In [4]:
import cv2
import numpy as np
import glob
import os
import shutil
from tqdm import tqdm

path ="/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/train"
dst="/kaggle/working/datasets_aug/badodd/images/train"
txt_source_directory="/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/labels/train"
new_directory ="/kaggle/working/datasets_aug/badodd/labels/train"
images = glob.glob(os.path.join(path, "*.jpg"))

def save_as(src,image_rgb,num):
    file_name, file_extension = os.path.splitext(os.path.basename(src))
    new_file_name = f"{file_name}_{num}{file_extension}"
    save_path = os.path.join(dst, new_file_name)
    cv2.imwrite(save_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
    
def save_label(jpg_source_path, num):
    # Extract the file name (excluding extension) from the JPG source path
    jpg_base_name = os.path.splitext(os.path.basename(jpg_source_path))[0]

    # Create the full path for the TXT source file
    txt_source_path = os.path.join(txt_source_directory, f"{jpg_base_name}.txt")

    # Check if the TXT file exists
    if os.path.exists(txt_source_path):
        # Create the new TXT file name with "_1" appended
        new_txt_base_name = f"{jpg_base_name}_{num}.txt"
        
        # Create the full path for the new TXT file in the new directory
        new_txt_destination_path = os.path.join(new_directory, new_txt_base_name)

        # Copy the TXT file to the new directory with the new name
        shutil.copy(txt_source_path, new_txt_destination_path)

#         print(f"TXT file saved as: {new_txt_destination_path}")
    else:
        raise FileNotFoundError(f"TXT file not found for: {jpg_source_path}")
    
for img in tqdm (images,desc="Processing Image"):
    # Extract the file name and extension
    image=cv2.imread(img)
    #original image
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    save_as(img,image_rgb,0)
    
    #1 brightening
    brightened_img = adjust_brightness(image_rgb, 1.6)
    save_as(img,brightened_img,1)
    
    #2 contrast
    contrasted_img = adjust_contrast(image_rgb, 1.5)
    save_as(img,contrasted_img,2)
    
    #3 denoise
    denoised_img = cv2.GaussianBlur(image_rgb, ksize=(5,5), sigmaX=1.5)
    save_as(img,denoised_img,3)
    
    #4 non-local blur
    non_local_img = cv2.fastNlMeansDenoisingColored(image, None, h=10, templateWindowSize=7, searchWindowSize=21)
    save_as(img,non_local_img,4)
    
    #5 greyscale imagegreat 
    grayscale_img = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    save_as(img, cv2.cvtColor(grayscale_img, cv2.COLOR_GRAY2RGB), 5)
    
    
    #generating all label
    for i in tqdm(range(6), desc ="Creating Labels"):
        save_label(img,i)
    
    
#copying test images
src_path = r"/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/test/"
dst_path = r"/kaggle/working/datasets_aug/badodd/images/test/"
shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

Processing Image:   0%|          | 12/5896 [00:36<4:57:51,  3.04s/it]


KeyboardInterrupt: 

We are stopping the above cell run with keyboard interrupt as it takes more than 5 hours for creating **Augmented-Dataset-For-DLE**

The above part generates a augmented version of the actual dataset which is 5x in size of the original dataset. We have considered various factors like brightness, contrast, denoising , grayscale and so on for the augmentation. From here on, for the later part of the notebook, we are using the Augmented Dataset we generated here. 

Augmented Dataset Name: **Augmented-Dataset-For-DLE**   
Augmented Dataset Link: https://www.kaggle.com/datasets/kabidhasan/augmented-dataset-for-dle   
N.B: Augmented dataset is accessible with view permission for the organizer personnels. To use our new augmented dataset, you need to go right sidebar, click add data, serach for the name **Augmented-Dataset-For-DLE**, and click + icon to add this dataset. 

## Creating Class Balanced Training and Validation Set From the Augmented Datasets

In [12]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [5]:
# delete all the files and directories recursively in the current working directory ...
!rm -rf *

In [6]:
# make directory ...

# !mkdir /kaggle/working/datasets
!mkdir /kaggle/working/BO
!mkdir /kaggle/working/BO/labels
!mkdir /kaggle/working/BO/labels/train
!mkdir /kaggle/working/BO/images
!mkdir /kaggle/working/BO/images/train
!mkdir /kaggle/working/BO/images/test

In [24]:
import shutil
src_path = r"/kaggle/input/augmented-dataset-for-dle/datasets/badodd/images/test/"
dst_path = r"/kaggle/working/BO/images/test/"
shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
print('Copied')

Copied


In [25]:
# This Python code converts a dataset in YOLO format into the COCO format. 
# The YOLO dataset contains images of datasets/badoddttles and the datasets/badoddunding datasets/badoddx annotations in the 
# YOLO format. The COCO format is a widely used format for object detection datasets.

# The input and output directories are specified in the code. The categories for 
# the COCO dataset are also defined, with only one category for "datasets/badoddttle". A dictionary for the COCO dataset is initialized with empty values for "info", "licenses", "images", and "annotations".

# The code then loops through each image in the input directory. The dimensions 
# of the image are extracted and added to the COCO dataset as an "image" dictionary, 
# including the file name and an ID. The datasets/badoddunding datasets/badoddx annotations for each image are 
# read from a text file with the same name as the image file, and the coordinates are 
# converted to the COCO format. The annotations are added to the COCO dataset as an 
# "annotation" dictionary, including an ID, image ID, category ID, datasets/badoddunding datasets/badoddx coordinates,
# area, and an "iscrowd" flag.

# The COCO dataset is saved as a JSON file in the output directory.

import json
import os
from PIL import Image
from tqdm import tqdm
# Set the paths for the input and output directories
input_img_dir = '/kaggle/input/augmented-dataset-for-dle/datasets/badodd/images/train/'
input_label_dir ='/kaggle/input/augmented-dataset-for-dle/datasets/badodd/labels/train/'
output_dir = '/kaggle/working/'

# Define the categories for the COCO dataset
categories = [
    {"id": 0, "name": "auto_rickshaw"},
    {"id": 1, "name": "bicycle"},
    {"id": 2, "name": "bus"},
    {"id": 3, "name": "car"},
    {"id": 4, "name": "cart_vehicle"},
    {"id": 5, "name": "construction_vehicle"},
    {"id": 6, "name": "motorbike"},
    {"id": 7, "name": "person"},
    {"id": 8, "name": "priority_vehicle"},
    {"id": 9, "name": "three_wheeler"},
    {"id": 10, "name": "train"},
    {"id": 11, "name": "truck"},
    {"id": 12, "name": "wheelchair"}
]


# Define the COCO dataset dictionary
coco_dataset = {
    "info": {},
    "licenses": [],
    "categories": categories,
    "images": [],
    "annotations": []
}

# Loop through the images in the input directory
for image_file in tqdm (os.listdir(input_img_dir), desc="Converting to COCO"):
    
    # Load the image and get its dimensions
    image_path = os.path.join(input_img_dir, image_file)
    image = Image.open(image_path)
    width, height = image.size
    
    # Add the image to the COCO dataset
    image_dict = {
        "id": hash(image_file.split('.')[0]),
        "width": width,
        "height": height,
        "file_name": image_file
    }
    coco_dataset["images"].append(image_dict)
    
    # Load the datasets/badoddunding datasets/badoddx annotations for the image
    with open(os.path.join(input_label_dir, f'{image_file.split(".")[0]}.txt')) as f:
        annotations = f.readlines()
    
    # Loop through the annotations and add them to the COCO dataset
    for ann in annotations:
        cat = int(ann.strip().split()[0])
        x, y, w, h = map(float, ann.strip().split()[1:])
        x_min, y_min = int((x - w / 2) * width), int((y - h / 2) * height)
        x_max, y_max = int((x + w / 2) * width), int((y + h / 2) * height)
        ann_dict = {
            "id": len(coco_dataset["annotations"]),
            "image_id": hash(image_file.split('.')[0]),
            "category_id": cat,
            "bdatasets/badoddx": [x_min, y_min, x_max - x_min, y_max - y_min],
            "area": (x_max - x_min) * (y_max - y_min),
            "iscrowd": 0
        }
        coco_dataset["annotations"].append(ann_dict)

# Save the COCO dataset to a JSON file
with open(os.path.join(output_dir, 'annotations.json'), 'w') as f:
    json.dump(coco_dataset, f)


Converting to COCO: 100%|██████████| 35376/35376 [11:55<00:00, 49.47it/s]


## Making Equally Distributed Class Validations

In [7]:
! pip install -q ultralytics pycocotools

In [8]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from tqdm.notebook import tqdm

In [ ]:
coco = COCO("/kaggle/working/annotations.json")

In [ ]:
ann_ids = coco.getAnnIds()
anns = coco.loadAnns(ann_ids)

cat_ids = [ann["category_id"] for ann in anns]
img_ids = [ann["image_id"] for ann in anns]

ann_ids = pd.Series(ann_ids)
cat_ids = pd.Series(cat_ids)
img_ids = pd.Series(img_ids)

In [ ]:
import os
from sklearn.model_selection import StratifiedGroupKFold

FOLDS = 5
SEED = 3000

sgkf = StratifiedGroupKFold(n_splits=FOLDS, shuffle=True, random_state=3000)

counts = cat_ids.value_counts()

print(f"Number of images: {len(img_ids.unique())}")

for cls, count in zip(counts.index, counts):
    print(f"Number of instances of class {cls}: {count}")


In [ ]:
folds = []
number_of_images = []
auto_rickshaw = []
bicycle = []
bus = []
car = []
cart_vehicle = []
construction_vehicle = []
motorbike = []
person = []
priority_vehicle = []
three_wheeler = []
train = []
truck = []
wheelchair = []


for fold, (_, val_idx) in enumerate(sgkf.split(ann_ids, cat_ids, img_ids)):
    folds.append(fold)
    val_ann_ids = ann_ids[val_idx]
    val_cat_ids = cat_ids[val_idx]
    val_img_ids = set(img_ids[val_idx])
    
    os.makedirs(f"/kaggle/working/BO/images/train/val_{fold}/")
    os.makedirs(f"/kaggle/working/BO/labels/train/val_{fold}/")
    
    for img in tqdm(coco.loadImgs(val_img_ids)):
        img_src = "/kaggle/input/augmented-dataset-for-dle/datasets/badodd/images/train/" + img["file_name"]
        img_dst = f"/kaggle/working/BO/images/train/val_{fold}/" + img["file_name"]
        shutil.copy(img_src, img_dst)

        label_src = "/kaggle/input/augmented-dataset-for-dle/datasets/badodd/labels/train/" + img["file_name"][:-4] + ".txt"
        label_dst = f"/kaggle/working/BO/labels/train/val_{fold}/" + img["file_name"][:-4] + ".txt"
        shutil.copy(label_src, label_dst)
    
    number_of_images.append(len(val_img_ids))
  
    auto_rickshaw.append(sum(val_cat_ids == 0))
    bicycle.append(sum(val_cat_ids == 1))
    bus.append(sum(val_cat_ids == 2))
    car.append(sum(val_cat_ids == 3))
    cart_vehicle.append(sum(val_cat_ids == 4))
    construction_vehicle.append(sum(val_cat_ids == 5))
    motorbike.append(sum(val_cat_ids == 6))
    person.append(sum(val_cat_ids == 7))
    priority_vehicle.append(sum(val_cat_ids == 8))
    three_wheeler.append(sum(val_cat_ids == 9))
    train.append(sum(val_cat_ids == 10))
    truck.append(sum(val_cat_ids == 11))
    wheelchair.append(sum(val_cat_ids == 12))

    
df = pd.DataFrame({
    "Fold": folds,
    "Number of Images": number_of_images,
    "auto_rickshaw": auto_rickshaw,
    "bicycle": bicycle,
    "bus": bus,
    "car": car,
    "cart_vehicle": cart_vehicle,
    "construction_vehicle": construction_vehicle,
    "motorbike": motorbike,
    "person": person,
    "priority_vehicle": priority_vehicle,
    "three_wheeler": three_wheeler,
    "train": train,
    "truck": truck,
    "wheelchair": wheelchair,
})

df.set_index("Fold")

#### Prepearing Train and Validation Data from KFold

In [ ]:
# Moving images
import os
import shutil

def move_src_to_dest(folder):
    source_folder = f'/kaggle/working/BO/images/train/{folder}'
    destination_folder = '/kaggle/working/BO/images/train_new'

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Move each file from source folder to destination folder and then delete from source folder
    for file in tqdm(files, desc="Moving files"):
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)
    
    shutil.rmtree(source_folder)
    print("All files moved and deleted successfully.")

for fold in [0, 2, 3, 4]:
    folder = f'val_{fold}'
    move_src_to_dest(folder)

In [ ]:
# Moving images
import os
import shutil

def move_src_to_dest(folder):
    source_folder = f'/kaggle/working/BO/images/train/{folder}'
    destination_folder = '/kaggle/working/BO/images/val_new'

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Move each file from source folder to destination folder and then delete from source folder
    for file in tqdm(files, desc="Moving files"):
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)
    shutil.rmtree(source_folder)
    print("All files moved and deleted successfully.")

for fold in [1]:
    folder = f'val_{fold}'
    move_src_to_dest(folder)

In [ ]:
# Moving labels
import os
import shutil

def move_src_to_dest(folder):
    source_folder = f'/kaggle/working/BO/labels/train/{folder}'
    destination_folder = '/kaggle/working/BO/labels/train_new'

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Move each file from source folder to destination folder and then delete from source folder
    for file in tqdm(files, desc="Moving files"):
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)
    
    shutil.rmtree(source_folder)
    print("All files moved and deleted successfully.")

for fold in [0, 2, 3, 4]:
    folder = f'val_{fold}'
    move_src_to_dest(folder)

In [ ]:
# Moving labels
import os
import shutil

def move_src_to_dest(folder):
    source_folder = f'/kaggle/working/BO/labels/train/{folder}'
    destination_folder = '/kaggle/working/BO/labels/val_new'

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Move each file from source folder to destination folder and then delete from source folder
    for file in tqdm(files, desc="Moving files"):
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.move(source_path, destination_path)

    shutil.rmtree(source_folder)
    print("All files moved and deleted successfully.")

for fold in [1]:
    folder = f'val_{fold}'
    move_src_to_dest(folder)

In [ ]:
# Delete train folder for images
def delete_image_train_folder():
    train_folder = '/kaggle/working/BO/images/train'
    shutil.rmtree(train_folder)
    print("Train folder deleted successfully.")

def rename_image_train_new():
    new_folder_name = '/kaggle/working/BO/images/train'
    os.rename('/kaggle/working/BO/images/train_new', new_folder_name)
    print("Renamed 'train_new' folder to 'train'.")


def rename_image_val_new():
    new_folder_name = '/kaggle/working/BO/images/val'
    os.rename('/kaggle/working/BO/images/val_new', new_folder_name)
    print("Renamed 'val_new' folder to 'val'.")
delete_image_train_folder()
rename_image_train_new()
rename_image_val_new()

In [ ]:
# Delete train folder for labels
def delete_label_train_folder():
    train_folder = '/kaggle/working/BO/labels/train'
    shutil.rmtree(train_folder)
    print("Train folder deleted successfully.")

def rename_label_train_new():
    new_folder_name = '/kaggle/working/BO/labels/train'
    os.rename('/kaggle/working/BO/labels/train_new', new_folder_name)
    print("Renamed 'train_new' folder to 'train'.")


def rename_label_val_new():
    new_folder_name = '/kaggle/working/BO/labels/val'
    os.rename('/kaggle/working/BO/labels/val_new', new_folder_name)
    print("Renamed 'val_new' folder to 'val'.")
delete_label_train_folder()
rename_label_train_new()
rename_label_val_new()

The above part generates a class balanced train-val splitted version of the augmented dataset which splitted the images and labels of the augmented dataset in train and val folder. From here on, for the later part of the notebook, we are using the train n val dataset we generated here. 

Train-Test Dataset Name: **BadODD_augmented_train_n_val**   
Augmented Dataset Link: https://www.kaggle.com/datasets/niazmohaimanabtahi/badodd-augmented-train-n-val   
N.B: Augmented dataset is accessible with view permission for the organizer personnels. To use our new augmented dataset, you need to go right sidebar, click add data, serach for the name **BadODD_augmented_train_n_val**, and click + icon to add this dataset. 

## Training Part

## The training is done in our personal gpu on CLI
#### The training is performed into two steps. On our first step, we have trained the pretrained yolov8x model on the dataset provided by the authority for 45 epochs with the batch size of 64. And then, we have made an augmented dataset from the provided dataset and trained the model acquired from the first step on the new dataset for 25 epochs with the batch size of 64. Inference is done with the output model of second step

## First step

In [2]:
# Clone the ultralytics repository
! git clone https://github.com/ultralytics/ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 21834, done.
remote: Counting objects: 100% (904/904), done.
remote: Compressing objects: 100% (597/597), done.
remote: Total 21834 (delta 572), reused 547 (delta 306), pack-reused 20930
Receiving objects: 100% (21834/21834), 13.18 MiB | 23.67 MiB/s, done.
Resolving deltas: 100% (15087/15087), done.


In [3]:
ls

ultralytics/


In [4]:
cd ultralytics/

/kaggle/working/ultralytics


In [5]:
! pip install -e .

Obtaining file:///kaggle/working/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.1.12-0.editable-py3-none-any.whl size=22649 sha256=60f087c07b1ca946e1eb724faf1269368efe55a2213f9cfc794b02379c45f398
  Stored in directory: /tmp/pip-ephem-wheel-cache-4856crwu/wheels/1b/eb/7d/853f2df0532389050170c8cd117b40e1fef8d125854a910fe6
Successfully built ultralytics


In [6]:
! export MKL_SERVICE_FORCE_INTEL=1

In [7]:
! yolo detect train data=/kaggle/input/journey-by-neuron-modelckpt/badodd_first_step.yaml model=yolov8x.pt epochs=45 imgsz=640 batch=64 device=0 pretrained=True optimizer=AdamW seed=0 lr0=1e-3 weight_decay=1e-4 val=False plots=True

100%|█████████████████████████████████████████| 131M/131M [00:00<00:00, 227MB/s]
Ultralytics YOLOv8.1.12 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/journey-by-neuron-modelckpt/badodd_first_step.yaml, epochs=45, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

## The output model ckpt is "/kaggle/input/journey-by-neuron-modelckpt/best_first_step.pt"

## Second step

In [8]:
! yolo detect train data=/kaggle/input/journey-by-neuron-modelckpt/badodd_second_step.yaml model=/kaggle/input/journey-by-neuron-modelckpt/best_first_step.pt epochs=25 imgsz=640 batch=64 device=0 pretrained=True optimizer=AdamW seed=0 lr0=1e-3 weight_decay=1e-4 val=True plots=True

Ultralytics YOLOv8.1.12 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/journey-by-neuron-modelckpt/best_first_step.pt, data=/kaggle/input/journey-by-neuron-modelckpt/badodd_second_step.yaml, epochs=25, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt

## The output model ckpt is "/kaggle/input/journey-by-neuron-modelckpt/best_second_step.pt"